# Task 6: Building conversational applications with the Converse API

In this notebook, you learn how to use the flexible Converse API to integrate external capabilities into conversational applications.

Certain conversational applications demand an adaptable sequence of calls to language models and various utilities depending on user input. The Converse API enables building such flexible dialogue agents.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio.

## Task 6.1: Environment setup

In this task, you set up your environment.

In [ ]:
#create a service client by name using the default session.
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## Task 6.2: Synergizing Reasoning and Acting in Language Models Framework

In this task, the Converse API enables integrating large language models with external capabilities to obtain additional information that results in more accurate and factual responses in a conversation.

The Converse framework allows conversational models like Mistral to generate natural language responses in an ongoing dialogue. Within these responses, you can configure markup like [tools] tags to trigger calls to external capabilities.

Large language models can generate both explanations for their reasoning and task-specific responses in an alternating fashion.

Producing reasoning explanations enables the models to infer, monitor, and revise action plans, and even handle unexpected scenarios. The action step allows the models to interface with and obtain information from external sources such as knowledge bases or environments.

In [ ]:
# imports

import os
import boto3

from botocore.exceptions import ClientError

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

from langchain.prompts import PromptTemplate

Next, you will build a simple calculator agent to add, subtract, multiply, and divide using the Converse API. The purpose is to demonstrate how to register the calculator functions as conversational capabilities that can be called through the Converse API. 
  

In [ ]:
# write a function to add, subtract, multiply, and divide two numbers

def calculator(oper1, oper2, operator):

    def add(a, b):
        return a+b

    def subtract(a, b):
        return a-b

    def multiply(a, b):
        return a*b

    def divide(a, b):
        if b==0:		
            return "Cannot divide by zero"
        else:
            return float(a/b)
        
    if operator == '+' or operator == 'add':
        return add(oper1, oper2)
    elif operator == '-' or operator == 'subtract':
        return subtract(oper1,oper2)
    elif operator == '*' or operator == 'multiply':
        return multiply(float(oper1), float(oper2))
    elif operator == '/' or operator == 'divide':
        return divide(oper1,oper2)
    else:
        return "Invalid operator"

Next, you extend the conversational assistant with access to factual information from Wikipedia.

The intent of the following code is to demonstrate:

- Register it as an additional capability with our Converse API
- Update the prompt templates to invoke this capability based on certain trigger phrases


In [ ]:
def wiki_search(query:str):
    api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=5000)
    tool = WikipediaQueryRun(api_wrapper=api_wrapper)
    result = tool.invoke(query)
    return result

In [ ]:
# Create a ReAct template that has multiple input variables
ReAct_prompt = PromptTemplate(
    input_variables=["tool_names", "input"], 
    template="""
Answer the following questions as best you can. 
You have access to the following tools:\n\n{tool_names}\n\n
Use the following format:\n\nQuestion: the input question you must answer\n
Thought: you should always think about what to do\n
Action: the action to take, should be one of {tool_names}\n
Action Input: the input to the action\n
Observation: the result of the action\n... 
(this Thought/Action/Action Input/Observation can repeat N times)\n
Thought: I now know the final answer\n
Final Answer: the final answer to the original input question\n
Question: {input}\n
Assistant:\n
"""
)

Next, you enable conversing with an Amazon Bedrock model to generate text responses.

In [ ]:
def generate_text(bedrock_client, model_id, tool_config, input_text):
    """Generates text using the supplied Amazon Bedrock model. If necessary,
    the function handles tool use requests and sends the result to the model.
    Args:
        bedrock_client: The Boto3 Bedrock runtime client.
        model_id (str): The Amazon Bedrock model ID.
        tool_config (dict): The tool configuration.
        input_text (str): The input text.
    Returns:
        Nothing.
    """

   # Create the initial message from the user input.
    messages = [{
        "role": "user",
        "content": [{"text": input_text}]
    }]

    response = bedrock_client.converse(
        modelId=model_id,
        messages=messages,
        toolConfig=tool_config
    )
    output_message = response['output']['message']
    messages.append(output_message)       
    
    stop_reason = response['stopReason']
    if stop_reason == "end_turn":
        # print the final response from the model.
        for content in output_message['content']:
            print(f"{content['text']}")
    else:
        print(f"Unexpected stop_reason: {stop_reason}")

Now, you define a tool configuration to register external capabilities that can be invoked within conversations with an Amazon Bedrock model.

In [ ]:
tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "wiki_search",
                "description": "Useful when you need to search the internet.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "query": {
                                "type": "string",
                                "description": "Your search query."
                            }
                        },
                        "required": [
                            "query"
                        ]
                    }
                }
            }
        },
        {
            "toolSpec": {
                "name": "calculator",
                "description": "Useful when you need to answer questions that involve adding, subtracting, multiplying, and dividing two numbers.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "oper1": {
                                "type": "integer",
                                "description": "first operand."
                            },
                            "oper2": {
                                "type": "integer",
                                "description": "second operand."
                            },
                            "operator": {
                                "type": "string",
                                "description": "operator."
                            }
                        },
                        "required": [
                            "oper1","oper2","operator"
                        ]
                    }
                }
            }
        }
    ]
}

Next, you orchestrate the end-to-end flow to have a conversation with the Amazon Bedrock model leveraging the integrated tools.

In [ ]:
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))
model_id = "mistral.mistral-large-2402-v1:0"

query="What is an Astronomical unit (AU)? Express the circumference of the earth in AU"
#query="What is Amazon SageMaker? What is launch year multiplied by 2"

# Pass in values to the input variables
input_text = ReAct_prompt.format(tool_names="""["wiki_search", "calculator"]""",
                                 input=query
     )

try:
    generate_text(bedrock_client, model_id, tool_config, input_text)
except ClientError as err:
    message = err.response['Error']['Message']
    print(f"A client error occured: {message}") 


You have successfully integrated Amazon Bedrock with custom capabilities by utilizing Converse API and tool framework.

### Try it yourself

- Change the prompts to your specific usecase and evaluate the output of different models.
- Play with the token length to understand the latency and responsiveness of the service.
- Apply different prompt engineering principles to get better outputs.

### Cleanup

You have completed this notebook. To move to the next part of the lab, do the following:

- Close this notebook file.
- Return to the lab session and continue with the **Conclusion**